<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=3537d205f1273072702b00dd43950e3a00b344ba0a4694838bc9673d1a5463b8
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


# 銘柄コードから株価情報を取得する関数

In [3]:
!pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=adca26cdacf4c094caae7da06b1d102cd0d4421dc1c0254111e08ad673a7597b
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


In [4]:
from yahoo_finance_api2 import share as yapi2
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    data = yapi2.Share(f'{code}.T').get_historical(
        period_type_value,
        period,
        yapi2.FREQUENCY_TYPE_DAY,
        1
    )

    df = pd.DataFrame(data)

    # タイムスタンプをDateTime型に変更
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

In [5]:
get_stock_data_yapi2(9501)

,Open,High,Low,Close,Volume
Date,,,,,
2013-05-15,517.0,612.0,504.0,513.0,699968100
2013-05-16,533.0,613.0,526.0,613.0,618650500
2013-05-17,633.0,656.0,607.0,626.0,505085200
2013-05-20,656.0,726.0,646.0,726.0,443322400
2013-05-21,810.0,841.0,770.0,815.0,549781900
...,...,...,...,...,...
2023-05-08,486.0,499.0,485.0,489.0,18300400
2023-05-09,490.0,496.0,487.0,496.0,12776300
2023-05-10,497.0,498.0,490.0,492.0,11458800


# JPX400の銘柄一覧を取得する関数

In [6]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_jpx400():
    """JPX400の銘柄コード一覧を取得する

    :return: JPX400の銘柄コード一覧
    """

    url = "https://indexes.nikkei.co.jp/nkave/archives/file/jpx_nikkei_index_400_weight_jp.csv"
    r = requests.get(url)
    with open('jpx_nikkei_index_400_weight_jp.csv', 'wb') as output:
        output.write(r.content)

    df = pd.read_csv("./jpx_nikkei_index_400_weight_jp.csv", encoding='shift_jis')
    # 不要なレコードとカラムを削除
    df = df.iloc[:-1]   # 最後の行は説明文のため除外
    df = df[['銘柄名', 'コード', '業種']]   # 必要なカラムのみにする
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

In [7]:
# JPX400の銘柄コードを取得
jpx400 = get_ticker_jpx400()
jpx400

,銘柄名,コード,業種
0,ニッスイ,1332,水産・農林業
1,ウエストホールディングス,1407,建設業
2,ショーボンドホールディングス,1414,建設業
3,ミライト・ワン,1417,建設業
4,ＩＮＰＥＸ,1605,鉱業
...,...,...,...
394,ニトリホールディングス,9843,小売業
395,ミスミグループ本社,9962,卸売業
396,ファーストリテイリング,9983,小売業
397,ソフトバンクグループ,9984,情報・通信業


In [18]:
# ニッスイの株価情報を取得
get_stock_data_yapi2(jpx400.loc[jpx400['銘柄名'] == 'ニッスイ', 'コード'].values[0])

,Open,High,Low,Close,Volume
Date,,,,,
2013-05-15,203.0,222.0,203.0,219.0,10037200
2013-05-16,221.0,225.0,203.0,206.0,7419700
2013-05-17,212.0,219.0,210.0,219.0,5122400
2013-05-20,226.0,232.0,225.0,230.0,6268800
2013-05-21,231.0,235.0,227.0,232.0,3023300
...,...,...,...,...,...
2023-05-08,591.0,599.0,591.0,596.0,1811000
2023-05-09,598.0,601.0,595.0,601.0,1110200
2023-05-10,602.0,602.0,593.0,596.0,1062600


# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [36]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
    
    close = df['Close']
    
    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)
    
    # パーフェクトオーダーかどうかの判定
    conditions = []
    if not reverse:
        # 上昇トレンド
        for i in range(len(mas) - 1):
            conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    else:
        # 下降トレンド
        for i in range(len(mas) - 1):
            conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
            
    result = all(conditions)

    return result

In [37]:
def filtered_ticker(df, min_volume=None, max_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    # 空のDataFrameを作成
    result_df = pd.DataFrame()

    # 各銘柄についてループ
    for index, row in df.iterrows():
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=1)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                stock_data = get_stock_data_yapi2(code, period_type='day', period=largest_mas*2)
            elif period_type == 'W':
                stock_data = get_stock_data_yapi2(code, period_type='week', period=largest_mas*2)
            elif period_type == 'M':
                stock_data = get_stock_data_yapi2(code, period_type='month', period=largest_mas*2)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        
        print(stock_data)

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue
        
        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        
        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        filtered_df = pd.DataFrame({
            '銘柄名': [row['銘柄名']],
            'コード': [code],
            '業種': [row['業種']],
            '出来高': [volume],
            '株価': [price],
        }, index=[index])

        # 結果をresult_dfに追加
        result_df = pd.concat([result_df, filtered_df])

    return result_df

In [38]:
# JPX400の銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_jpx400 = filtered_ticker(jpx400, min_volume=200000, max_price=5000)
filtered_jpx400

             Open   High    Low  Close   Volume
Date                                           
2023-05-12  588.0  624.0  584.0  622.0  5378900
              Open    High     Low   Close  Volume
Date                                              
2023-05-12  3025.0  3070.0  3005.0  3030.0  187100
              Open    High     Low   Close  Volume
Date                                              
2023-05-12  5790.0  5790.0  5710.0  5760.0  148700
              Open    High     Low   Close  Volume
Date                                              
2023-05-12  1692.0  1707.0  1684.0  1707.0  394800
              Open    High     Low   Close    Volume
Date                                                
2023-05-12  1476.0  1492.0  1454.0  1460.0  10555900
             Open   High    Low  Close   Volume
Date                                           
2023-05-12  911.0  995.0  883.0  969.0  1638500
              Open    High     Low   Close   Volume
Date                                      

KeyboardInterrupt: ignored

In [39]:
# 株価情報がパーフェクトオーダーかどうかの判断
filtered_perfect_order_jpx400 = filtered_ticker(jpx400, min_volume=200000, max_price=5000, perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_jpx400

             Open   High    Low  Close   Volume
Date                                           
2023-03-15  544.0  549.0  543.0  547.0  1395800
2023-03-16  538.0  541.0  536.0  538.0  1897500
2023-03-17  539.0  542.0  537.0  537.0  2387500
2023-03-20  537.0  539.0  534.0  535.0  1641300
2023-03-22  542.0  544.0  540.0  542.0  1545900
2023-03-23  542.0  545.0  541.0  544.0  1379700
2023-03-24  544.0  550.0  544.0  549.0  1966400
2023-03-27  553.0  554.0  549.0  549.0  1338500
2023-03-28  550.0  551.0  545.0  548.0  1864800
2023-03-29  551.0  555.0  549.0  555.0  2973400
2023-03-30  543.0  545.0  538.0  540.0  2067500
2023-03-31  545.0  547.0  542.0  543.0  1954000
2023-04-03  546.0  551.0  543.0  551.0  1279600
2023-04-04  552.0  552.0  547.0  551.0  1026100
2023-04-05  549.0  552.0  543.0  545.0  1727400
2023-04-06  545.0  552.0  543.0  547.0  1667000
2023-04-07  550.0  551.0  548.0  548.0  1419300
2023-04-10  550.0  552.0  549.0  549.0   980000
2023-04-11  549.0  556.0  548.0  554.0  

TypeError: ignored